In [1]:
cd ..

/Users/mig/Desktop/pitt/projects/Group-Project-2140/cord_ir


In [2]:
from search.elastic_index_reader import IndexReader
from tqdm.notebook import trange, tqdm
import math

In [3]:
import xml.etree.ElementTree as ET
queryTree = ET.parse('../data/2020-07-16/eval/topics-rnd5.xml')

In [4]:
queryRoot = queryTree.getroot()
queries = []
for child in queryRoot:
    query = {
        'queryNo': child.attrib['number'],
        'query': child.find('query').text,
        'question': child.find('question').text,
        'narrative': child.find('narrative').text
    }
    queries.append(query)

In [5]:
reader = IndexReader()
for query in queries:
    query['query_tokens'] = [t['token'] for t in reader.tokenize(query['query'])['tokens']]
    query['question_tokens'] = [t['token'] for t in reader.tokenize(query['question'])['tokens']]
    query['narrative_tokens'] = [t['token'] for t in reader.tokenize(query['narrative'])['tokens']]

/opt/homebrew/Caskroom/miniforge/base/envs/info-retri/lib/python3.9/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [6]:
def getRetrievalResults(queries, field):
    results = {}
    for query in tqdm(queries):
        res = reader.search("cord_test", query[field], size=200, fields=[], highlight=False)
        results[query['queryNo']] = res['hits']['hits']
    return results

In [7]:
queryFields = ['query', 'question', 'narrative']
results = {f: getRetrievalResults(queries, f) for f in queryFields}

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [8]:
# load the judgments
judgments = {q['queryNo']: [] for q in queries}
with open('../data/2020-07-16/eval/qrels-covid_d5_j0.5-5.txt', 'r') as qrels:
    for line in qrels:
        [topicId, iteration, cordId, judgment] = line.strip('\n').split(' ')
        judgments[topicId].append({
            'iteration': iteration,
            'cordId': cordId,
            'judgment': judgment
        })

In [9]:
for field in queryFields:
    with open('../data/2020-07-16/eval/%s-result.txt'%field, 'w') as res_file:
        fieldResults = results[field]
        for query in tqdm(queries):
            for (index, result) in enumerate(fieldResults[query['queryNo']]):
                line = ' '.join([query['queryNo'], 'Q0', result['_id'], str(index + 1), str(result['_score']), field])
                res_file.write(line + '\n')

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [10]:
# use trec_eval to get evaluation
!../data/2020-07-16/eval/trec_eval ../data/2020-07-16/eval/qrels-covid_d5_j0.5-5.txt ../data/2020-07-16/eval/query-result.txt
!../data/2020-07-16/eval/trec_eval ../data/2020-07-16/eval/qrels-covid_d5_j0.5-5.txt ../data/2020-07-16/eval/question-result.txt
!../data/2020-07-16/eval/trec_eval ../data/2020-07-16/eval/qrels-covid_d5_j0.5-5.txt ../data/2020-07-16/eval/narrative-result.txt

runid                 	all	query
num_q                 	all	50
num_ret               	all	10000
num_rel               	all	26664
num_rel_ret           	all	3774
map                   	all	0.0938
gm_map                	all	0.0542
Rprec                 	all	0.1514
bpref                 	all	0.1417
recip_rank            	all	0.8236
iprec_at_recall_0.00  	all	0.8466
iprec_at_recall_0.10  	all	0.4202
iprec_at_recall_0.20  	all	0.1483
iprec_at_recall_0.30  	all	0.0284
iprec_at_recall_0.40  	all	0.0000
iprec_at_recall_0.50  	all	0.0000
iprec_at_recall_0.60  	all	0.0000
iprec_at_recall_0.70  	all	0.0000
iprec_at_recall_0.80  	all	0.0000
iprec_at_recall_0.90  	all	0.0000
iprec_at_recall_1.00  	all	0.0000
P_5                   	all	0.6640
P_10                  	all	0.6180
P_15                  	all	0.6067
P_20                  	all	0.5840
P_30                  	all	0.5640
P_100                 	all	0.4620
P_200                 	all	0.3774
P_500                 	all	0.1510
P_1000                	

In [35]:
# calculate in Python
ndcg_eval_at = [3, 5, 10, 15, 20, 25, 30]
recall_eval_at = [5, 10, 20, 50, 100, 150]
def getEvaluation(results, judgments):
    queryEval = []
    for queryNo in results:
        # doc to relevant score
        relMap = {}
        # number of docs in each relevant score
        relCount = {}
        totalRel = 0
        for j in judgments[queryNo]:
            score = int(j['judgment'])
            relMap[j['cordId']] = score
            relCount[score] = relCount.get(score, 0) + 1
            if score > 0:
                totalRel += 1
        truePositive = 0
        falsePositive = 0
        precisions = []
        reciprocalRank = 0
        cumulativeGain = 0
        discountedCumulativeGain = 0
        idealDCG = 0
        dcgAt = {}
        idcgAt = {}
        ndcgAt = {}
        index = 1
        recallAt = {}
        for score in sorted(relCount.keys(), reverse=True):
            for i in range(relCount[score]):
                idealDCG += (2 ** score - 1) / (math.log2(1 + index))
                if index in ndcg_eval_at:
                    idcgAt[index] = idealDCG
                index += 1
        for (index, doc) in enumerate(results[queryNo]):
            docId = doc['_id']
            cumulativeGain += relMap.get(docId, 0)
            discountedCumulativeGain += (2 ** (relMap.get(docId, 0)) - 1) / (math.log2(1 + (1 + i)))
            if relMap.get(docId, 0) > 0:
                truePositive += 1
                # recall increase
                precisions.append(truePositive / (truePositive + falsePositive))
                if reciprocalRank == 0:
                    reciprocalRank = 1 / (index+1)
            else:
                falsePositive += 1
            if index + 1 in ndcg_eval_at and (index + 1) in idcgAt:
                dcgAt[index + 1] = discountedCumulativeGain
                ndcgAt[index + 1] = dcgAt[index + 1] / idcgAt[index + 1]
            if index + 1 in recall_eval_at:
                recallAt[index + 1] = truePositive / totalRel
        queryEval.append({
            'AveragePrecision': sum(precisions) / totalRel,
            'ReciprocalRank': reciprocalRank,
            'CG': cumulativeGain,
            'DCG': discountedCumulativeGain,
            'IDCG': idealDCG,
            'nDCG': discountedCumulativeGain / idealDCG,
            'nDCGAt': ndcgAt,
            'recallAt': recallAt
        })
    return {
        'MeanAveragePrecisions': sum(map(lambda e: e['AveragePrecision'], queryEval)) / len(queryEval),
        'MeanReciprocalRank': sum(map(lambda e: e['ReciprocalRank'], queryEval)) / len(queryEval),
        'AverageNDCG': sum(map(lambda e: e['nDCG'], queryEval)) / len(queryEval),
        'AverageNDCGAt': {k: sum(map(lambda e: e['nDCGAt'][k], queryEval)) / len(queryEval) for k in ndcg_eval_at},
        'AverageRecallAt': {k: sum(map(lambda e: e['recallAt'][k], queryEval)) / len(queryEval) for k in recall_eval_at}
    }
            

In [12]:
getEvaluation(results['query'], judgments)

{'MeanAveragePrecisions': 0.0938413425805633,
 'MeanReciprocalRank': 0.8136031746031747,
 'AverageNDCG': 0.14186232674604926,
 'AverageNDCGAt': {3: 0.13364055103725428,
  5: 0.1418463555798589,
  10: 0.1680738131354743,
  15: 0.18464160353165723,
  20: 0.19572503874359487,
  25: 0.20166118601071548,
  30: 0.21199265201748066},
 'AverageRecallAt': {5: 0.007571936093659915,
  10: 0.014347370761125028,
  20: 0.025675868223038946,
  50: 0.05593293689518463,
  100: 0.09603243791764829,
  150: 0.12785061256462518}}

In [13]:
getEvaluation(results['question'], judgments)

{'MeanAveragePrecisions': 0.05846350714788746,
 'MeanReciprocalRank': 0.6930461107519932,
 'AverageNDCG': 0.10352932326145607,
 'AverageNDCGAt': {3: 0.09875194087296979,
  5: 0.11636770250055294,
  10: 0.13835908558004478,
  15: 0.1488085799127936,
  20: 0.15100867206756832,
  25: 0.15061045685942806,
  30: 0.15584868010479463},
 'AverageRecallAt': {5: 0.005727560253085045,
  10: 0.011274666322520223,
  20: 0.019611381677786005,
  50: 0.04029976874013658,
  100: 0.06874504517191328,
  150: 0.09178983181898402}}

In [14]:
getEvaluation(results['narrative'], judgments)

{'MeanAveragePrecisions': 0.0529449798520697,
 'MeanReciprocalRank': 0.6434796152314557,
 'AverageNDCG': 0.10658332315533475,
 'AverageNDCGAt': {3: 0.09286698794380803,
  5: 0.09907587042444932,
  10: 0.12023696099287039,
  15: 0.1276837496201119,
  20: 0.1381839021514124,
  25: 0.1423471423387641,
  30: 0.14730348405985721},
 'AverageRecallAt': {5: 0.005375328395812915,
  10: 0.00957656146663195,
  20: 0.01846469816821065,
  50: 0.03979425169564572,
  100: 0.06755924917514011,
  150: 0.08948080605363223}}

In [26]:
# ML Part 
from search.ml_rank import MlRanker
from search.data_loader import DataLoader
ranker = MlRanker("../data/models/ranker.joblib",
                  '../data/models/tfidf.joblib',
                  "../data/models/docMatrix.joblib")
loader = DataLoader('../data/2020-07-16')
loader.load_metadata_mappings(loader.load_metadata())

In [27]:
def getMlRetrievalResults(queries, field):
    results = {}
    for query in tqdm(queries):
        res = reader.search("cord_test", query[field], size=400, fields=[], highlight=False)
        results[query['queryNo']] = ranker.rank(query[field], res['hits']['hits'], loader)[:200]
#         results[query['queryNo']] = ranker.whole_rank(query[field], loader, size=3000)
    return results
queryFields = ['query', 'question', 'narrative']
mlResults = {f: getMlRetrievalResults(queries, f) for f in queryFields}

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [36]:
getEvaluation(mlResults['query'], judgments)

{'MeanAveragePrecisions': 0.09754285722689846,
 'MeanReciprocalRank': 0.8740197483675743,
 'AverageNDCG': 0.15104653827727005,
 'AverageNDCGAt': {3: 0.16892479481067557,
  5: 0.18782455577724325,
  10: 0.21788512612649186,
  15: 0.22770403153041194,
  20: 0.2417994182563755,
  25: 0.2476971871220725,
  30: 0.24971336721201381},
 'AverageRecallAt': {5: 0.009647913174943519,
  10: 0.01687878994647765,
  20: 0.029313361404363896,
  50: 0.05986309074625796,
  100: 0.09821988546152953,
  150: 0.1298758285001646}}

In [37]:
getEvaluation(mlResults['question'], judgments)

{'MeanAveragePrecisions': 0.07524073913386099,
 'MeanReciprocalRank': 0.8668431372549018,
 'AverageNDCG': 0.12799153666405216,
 'AverageNDCGAt': {3: 0.1590133984955591,
  5: 0.18194284465784655,
  10: 0.2045261334924703,
  15: 0.22129992802537024,
  20: 0.2326832815309929,
  25: 0.23865599572551563,
  30: 0.2424065307072679},
 'AverageRecallAt': {5: 0.0089174591424478,
  10: 0.0155464485399609,
  20: 0.027169630125022604,
  50: 0.052452607077066116,
  100: 0.08294633970464638,
  150: 0.10545998922417661}}

In [38]:
getEvaluation(mlResults['narrative'], judgments)

{'MeanAveragePrecisions': 0.07335848683946339,
 'MeanReciprocalRank': 0.8372222222222223,
 'AverageNDCG': 0.12672902113658038,
 'AverageNDCGAt': {3: 0.1344476147594284,
  5: 0.1598113561731201,
  10: 0.1952501663401587,
  15: 0.20411586099207513,
  20: 0.2178886029231166,
  25: 0.22479251984292542,
  30: 0.22801644403485963},
 'AverageRecallAt': {5: 0.00809828584854587,
  10: 0.014198610575236983,
  20: 0.024856629988496956,
  50: 0.049210911679684194,
  100: 0.08249535002786873,
  150: 0.10796258061617346}}

In [39]:
def get_diff(res1, res2):
    res = {}
    for key in res1.keys():
        if isinstance(res1[key], dict):
            res[key] = {}
            for key2 in res1[key].keys():
                res[key][key2] = str(((res2[key][key2] - res1[key][key2]) / res1[key][key2]) * 100) + '%'
        else:
            res[key] = str(((res2[key] - res1[key]) / res1[key]) * 100) + '%'
    return res

In [40]:
get_diff(getEvaluation(results['query'], judgments), getEvaluation(mlResults['query'], judgments))

{'MeanAveragePrecisions': '3.94443914009157%',
 'MeanReciprocalRank': '7.425803592011198%',
 'AverageNDCG': '6.474031366806522%',
 'AverageNDCGAt': {3: '26.40234831386267%',
  5: '32.41408636085742%',
  10: '29.63656982713163%',
  15: '23.322169638421467%',
  20: '23.540360399743903%',
  25: '22.82838954884995%',
  30: '17.793406910831393%'},
 'AverageRecallAt': {5: '27.416727447314926%',
  10: '17.64378454770013%',
  20: '14.166972465067525%',
  50: '7.026546555991203%',
  100: '2.2778215270938618%',
  150: '1.5840486759621322%'}}

In [41]:
get_diff(getEvaluation(results['question'], judgments), getEvaluation(mlResults['question'], judgments))

{'MeanAveragePrecisions': '28.696930451904596%',
 'MeanReciprocalRank': '25.077267415054%',
 'AverageNDCG': '23.628294508231722%',
 'AverageNDCGAt': {3: '61.02306151137529%',
  5: '56.35166867454655%',
  10: '47.82269818785841%',
  15: '48.71449492701215%',
  20: '54.08603912951408%',
  25: '58.45911412928299%',
  30: '55.53967511580507%'},
 'AverageRecallAt': {5: '55.693851280649106%',
  10: '37.888324986684005%',
  20: '38.54011191775385%',
  50: '30.156099443880713%',
  100: '20.65791723202653%',
  150: '14.892888606824226%'}}

In [42]:
get_diff(getEvaluation(results['narrative'], judgments), getEvaluation(mlResults['narrative'], judgments))

{'MeanAveragePrecisions': '38.55607659957527%',
 'MeanReciprocalRank': '30.108585012608142%',
 'AverageNDCG': '18.90136034873415%',
 'AverageNDCGAt': {3: '44.77438941034675%',
  5: '61.301995620603556%',
  10: '62.38780881341165%',
  15: '59.86048467355173%',
  20: '57.68016355795863%',
  25: '57.918533628131506%',
  30: '54.793653042316684%'},
 'AverageRecallAt': {5: '50.65657857953364%',
  10: '48.264182553517266%',
  20: '34.617039293341065%',
  50: '23.663367403058484%',
  100: '22.108151045326714%',
  150: '20.65445694740812%'}}